<a href="https://colab.research.google.com/github/Luckynirwan12/Web-Crawling/blob/main/Web_Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Web **Crawling**

In [8]:
!pip install requests-html
!pip install requests
!pip install bs4
!pip install lxml_html_clean

In [17]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from collections import deque
from urllib.parse import urlsplit
from requests_html import HTMLSession
import threading
import time
import urllib3
from tqdm import tqdm

In [10]:
#user_url = "https://www.thapar.edu/sitemap.xml"
#url = ["https://www.thapar.edu"]
EMAIL_REGEX = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""

In [19]:
# Disable SSL warnings (not recommended in production)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Function to extract all URLs from sitemap XML
def get_urls_of_xml(xml_url):
    r = requests.get(xml_url, verify=False)       # Fetch sitemap, ignore SSL verification
    xml = r.text                                  # Get raw XML content
    soup = BeautifulSoup(xml, 'xml')              # Parse XML properly using 'xml' parser
    links_arr = []                                # List to store URLs

    for link in soup.findAll('loc'):              # Look for all <loc> tags
        linkstr = link.get_text(strip=True)       # Extract text inside <loc>
        links_arr.append(linkstr)                 # Add to the list

    return links_arr                              # Return all collected URLs

In [20]:
# Call the function with the sitemap URL
links_data_arr = get_urls_of_xml("https://thapar.edu/sitemap.xml")

# (Optional) Print few links to confirm it's working
print(links_data_arr[:5])  # Show first 5 URLs

['http://thapar.edu/', 'http://thapar.edu/programmes', 'http://thapar.edu/admissions', 'http://thapar.edu/aboutus', 'http://thapar.edu/students']


/tmp/ipython-input-19-726831832.py:11: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  for link in soup.findAll('loc'):              # Look for all <loc> tags


In [ ]:
session = HTMLSession()  # Create a session for advanced browsing
email = set()            # Store unique emails (set avoids duplicates)

for i in tqdm(links_data_arr):
    try:
        # Use session.get with verify=False to bypass SSL check
        r = session.get(i, verify=False)

        # Extract emails from the page
        for re_match in re.finditer(EMAIL_REGEX, r.html.raw_html.decode()):
            email.add(((re_match.group())).replace("-", ""))

    except Exception as e:
        print(f"❌ Error while processing {i}: {e}")

 23%|██▎       | 1014/4342 [14:36<36:44,  1.51it/s]

❌ Error while processing http://thapar.edu/AnnexureI.htm: 'utf-8' codec can't decode byte 0xa0 in position 721: invalid start byte


 37%|███▋      | 1615/4342 [25:44<36:18,  1.25it/s]

❌ Error while processing http://lmtsom.thapar.edu: HTTPConnectionPool(host='lmtsom.thapar.edu', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79a8f57dab90>: Failed to establish a new connection: [Errno -2] Name or service not known'))


 38%|███▊      | 1651/4342 [26:18<1:06:06,  1.47s/it]

...

 88%|████████▊ | 3811/4342 [1:03:02<05:56,  1.49it/s]

❌ Error while processing http://www.thapar.edu/upload/files/ClientAuth_CA.scc: 'utf-8' codec can't decode byte 0x82 in position 1: invalid start byte


 88%|████████▊ | 3818/4342 [1:03:07<04:37,  1.88it/s]

❌ Error while processing http://www.thapar.edu/pages/event/during-and-after-covid-193: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


 98%|█████████▊| 4254/4342 [1:10:24<02:04,  1.42s/it]

In [25]:
# Convert the set of emails into a pandas DataFrame
df = pd.DataFrame(email, columns=["Email"])

# Save the DataFrame as a CSV file named 'Emails.csv' without row index
df.to_csv("Emails.csv", index=False)

# Print a success message when the CSV file is saved
print("✅ Emails saved to Emails.csv")


✅ Emails saved to Emails.csv
